# Запускайте отсюда, чтобы выполнить полный цикл обучения

In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U scikit-learn==1.1.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from tensorflow import config
import keras
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

2024-03-12 11:52:16.888400: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-12 11:52:16.888787: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 11:52:16.892021: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 11:52:16.924044: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 11:52:17.385553: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [5]:
df = pd.read_csv('train.csv')

In [6]:
# Удаление ответа из тестовой выборки
X = df.drop('is_canceled', axis=1)
y = df['is_canceled']

In [7]:
# Разделение на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Предварительная обработка данных
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns
categories = []
for column in categorical_features:
    cat_arr = pd.Categorical(X_train[column]).categories
    categories.append(cat_arr.tolist())
encoder =OneHotEncoder(handle_unknown='ignore', categories=categories)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', encoder)
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = pd.DataFrame.sparse.from_spmatrix(preprocessor.fit_transform(X_train))
X_test = pd.DataFrame.sparse.from_spmatrix(preprocessor.transform(X_test))
print(X_train.head())

       0         1         2         3         4         5        6     \
0  2.268043 -0.221621  0.797619 -0.092391 -0.932961  0.256486 -1.44993   
1 -0.748929 -0.221621 -0.816334 -0.320037 -0.932961 -0.265312  0.24612   
2 -0.777662  1.194899  -1.54995 -0.661506  1.071494 -0.265312  0.24612   
3 -0.174268 -0.221621  1.164427   0.70437  1.071494 -0.265312  0.24612   
4  0.668569 -0.221621  0.137365 -0.320037 -0.932961  0.256486 -1.44993   

       7         8         9     ...  1096  1097  1098  1099  1100  1101  \
0 -0.261927 -0.090203 -0.188395  ...     0     0     0     0     0     0   
1 -0.261927 -0.090203 -0.188395  ...     0     0     0     0     0     0   
2 -0.261927 -0.090203 -0.188395  ...     0     0     0     0     0     0   
3 -0.261927 -0.090203 -0.188395  ...     0     0     0     0     0     0   
4 -0.261927 -0.090203 -0.188395  ...     0     0     0     0     0     0   

   1102  1103  1104  1105  
0     0     0     0     0  
1     0     0     0     0  
2     0     0 

In [9]:
# Создание модели нейронной сети
model = Sequential([
    Input((X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
gpus = config.list_logical_devices('GPU')
if gpus:
    device(gpus[0].name)


2024-03-12 11:52:18.227600: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-12 11:52:18.228139: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
# Обучение модели
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=128)

Epoch 1/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7509 - loss: 0.5082 - val_accuracy: 0.8618 - val_loss: 0.3166
Epoch 2/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - accuracy: 0.8683 - loss: 0.2997 - val_accuracy: 0.8867 - val_loss: 0.2708
Epoch 3/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - accuracy: 0.8949 - loss: 0.2469 - val_accuracy: 0.8877 - val_loss: 0.2661
Epoch 4/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.9109 - loss: 0.2128 - val_accuracy: 0.9013 - val_loss: 0.2383
Epoch 5/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - accuracy: 0.9338 - loss: 0.1717 - val_accuracy: 0.9066 - val_loss: 0.2269
Epoch 6/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9460 - loss: 0.1396 - val_accuracy: 0.9077 - val_loss: 0.2269
Epoch 7/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step - accuracy: 0.9577 - loss: 0.1153 - val_accuracy: 0.9059 - val_loss: 0.2410
Epoch 8/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - accuracy: 0.9653 - loss: 0.0950 - val_

In [12]:
### Оценка модели на тестовых данных
model.evaluate(X_test, y_test)

279/279 ━━━━━━━━━━━━━━━━━━━━ 0s 318us/step - accuracy: 0.9152 - loss: 0.2819


[0.2660931944847107, 0.9180107712745667]

## Очень важный момент, категории, определенные OneHotEncoder должны сохранятся и использоваться в предобработке всегда, так как они определяют кол-во столбцов после предобработки данных, а оно связано с InputLayer нейросети

In [13]:
with open("categories.json", 'w') as f:
    json.dump(categories, f, indent=2) 

## Прогнозирование.
### раскомментируйте и запускайте отсюда, чтобы не обучать модель, а использовать предобученную на предыдущих шагах

In [14]:
#раскомментируйте и запускайте отсюда, чтобы не обучать модель, а использовать предобученную на предыдущих шагах
#model = keras.models.load_model('hotel_model.keras')
#model.load_weights('hotel_model.keras')
#categories=[]
#with open("categories.json", 'r') as f:
#    categories = json.load(f)
test = pd.read_csv('test.csv')


In [15]:
encoder =OneHotEncoder(handle_unknown='ignore', categories=categories)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', encoder)
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

test_processed = pd.DataFrame.sparse.from_spmatrix(preprocessor.fit_transform(test))
print(test_processed.head)

<bound method NDFrame.head of            0         1         2         3         4         5         6     \
0     -0.462861 -0.232068  0.059016 -1.109345 -0.928225 -0.278773 -1.471895   
1     -0.778393 -1.636235  1.182209  0.934735  1.073837   1.31727 -1.471895   
2       0.56262  1.172099 -0.989298 -1.109345  1.073837  0.253241  0.199825   
3      1.412805 -0.232068  0.583172  0.934735  0.072806  0.785255  0.199825   
4       2.28052 -0.232068  1.182209  0.480495  0.072806  0.253241  0.199825   
...         ...       ...       ...       ...       ...       ...       ...   
23520  0.474972 -0.232068 -0.390262  0.821175  0.072806   1.31727  0.199825   
23521 -1.067632 -0.232068  1.331968 -1.677145 -0.928225  0.253241 -1.471895   
23522 -1.076396 -0.232068 -0.465141  0.026255  0.072806 -0.810787 -1.471895   
23523  1.631925  1.172099 -1.288816 -1.222905  1.073837   1.31727  0.199825   
23524  0.168204 -0.232068  0.358534 -1.677145  1.073837   1.31727  0.199825   

           7         

In [16]:
result = model.predict(test_processed)
print(result)
#assert np.allclose(model.predict(test_processed), loaded_model.predict(test_processed))

736/736 ━━━━━━━━━━━━━━━━━━━━ 0s 375us/step
[[4.5695631e-03]
 [5.3605872e-06]
 [9.9730116e-01]
 ...
 [1.1782342e-01]
 [9.9973726e-01]
 [6.1542571e-02]]


In [17]:
result = pd.DataFrame(np.array([1 if yi > 0.5 else 0 for yi in result]), columns=['is_canceled'])
result.reset_index(inplace= True )
print(result)
result.to_csv('example_result.csv', sep=',', index=False, header=True, encoding='utf-8')

       index  is_canceled
0          0            0
1          1            0
2          2            1
3          3            0
4          4            1
...      ...          ...
23520  23520            0
23521  23521            0
23522  23522            0
23523  23523            1
23524  23524            0

[23525 rows x 2 columns]


In [18]:
model.save('hotel_model.keras')